### Introducing Knowledge Graphs
In complex fields, such as science and medicine, the sheer amount of data and literature available on specific topics is hard to overstate. The same goes for knowledge management in established companies and industries where, over time, institutional knowledge in the form of textual information builds up, becoming too large to sensibly disseminate. In both of these case, a knowledge graph may help to alleviate issues associated with too much disparate information.

The quality of text data has a large impact on the preparation of data for knowledge graph ingestion.